<center>
    <p style="text-align:center">
        <img alt="arize llama-index logos" src="https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/arize_llamaindex.png" width="400">
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating and Improving a LlamaIndex Search and Retrieval Application using Milvus as a Vector Store</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LlamaIndex. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Load pre-built knowledge data into a Milvus Lite vector store
- Download a pre-indexed knowledge base of the Arize documentation and run a LlamaIndex application
- Visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using cosine similarity, Euclidean distance, and LLM-assisted ranking metrics

Parts of this notebook require an [OpenAI API key](https://platform.openai.com/account/api-keys) to run. If you don't have an OpenAI key, you can still run Phoenix by skipping cells preceded by the 💭 emoji.


## Chatbot Architecture

Your chatbot was built using LlamaIndex's low-level API. The architecture of your chatbot is shown below and can be explained in five steps.

![llama-index chatbot architecture](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/llama_index_chatbot_architecture.png)

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to the OpenAI embeddings API to embed the user query using the `text-embedding-ada-002` model.
1. `llama_index.retrievers.RetrieverQueryEngine` does a similarity search against the entries of your index knowledge base for the two most similar pieces of context by cosine similarity.
1. `llama_index.indices.query.ResponseSynthesizer` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI chat completions API with the `gpt-3.5-turbo`.
2. The response is returned to the user.

Phoenix makes your search and retrieval system *observable* by capturing the inputs and outputs of these steps for analysis, including:

- your query embeddings
- the retrieved context and similarity scores for each query
- the generated response that is return to the user

With that overview in mind, let's dive into the notebook.

### Setup

In [1]:
!pip install -q gcsfs llama-index milvus "arize-phoenix[experimental]"
import phoenix as px

In [2]:
!pip install pymilvus
!pip install --upgrade --force-reinstall grpcio==1.56.0

  Obtaining dependency information for grpcio==1.56.0 from https://files.pythonhosted.org/packages/d2/62/b2da52d8de3e7c785613cda521e405d2fe85da5728c7c84fd8ea0fce522e/grpcio-1.56.0-cp38-cp38-macosx_10_10_universal2.whl.metadata
  Using cached grpcio-1.56.0-cp38-cp38-macosx_10_10_universal2.whl.metadata (4.0 kB)
Using cached grpcio-1.56.0-cp38-cp38-macosx_10_10_universal2.whl (8.9 MB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.56.0
    Uninstalling grpcio-1.56.0:
      Successfully uninstalled grpcio-1.56.0


## Restart runtime after pip install! Not restarting might cause issues!


Import the necessary libraries

In [3]:
import json
import os
import textwrap
import getpass
import urllib.request
from datetime import timedelta


import numpy as np
import openai
import pandas as pd
from gcsfs import GCSFileSystem
from IPython.display import YouTubeVideo
from langchain.chat_models import ChatOpenAI
from llama_index import (
    LLMPredictor,
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)
from llama_index.callbacks import CallbackManager, OpenInferenceCallbackHandler
from llama_index.callbacks.open_inference_callback import as_dataframe
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.vector_stores import MilvusVectorStore
from milvus import default_server, MilvusServer, debug_server

from phoenix.experimental.evals.retrievals import (
    classify_relevance,
    compute_precisions_at_k,
)

from pymilvus import (
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    connections,
    utility
)



pd.set_option("display.max_colwidth", 1000)

And set up our openai api key

In [4]:
openai_api_key = getpass.getpass(prompt="Enter your OpenAI API key: ")
assert openai_api_key != "", "❌ Please set your OpenAI API key"
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API key:  ········


For this example in particular we'll be using a json file to generate the necessary data to build our vector store

In [5]:
url = "http://storage.googleapis.com/arize-assets/xander/milvus-workshop/milvus_dataset.json"

with urllib.request.urlopen(url) as response:
    buffer = response.read()
    data = json.loads(buffer.decode("utf-8"))
    rows = data['rows']



In [6]:
import logging, sys
logging.disable(sys.maxsize)

### Milvus lite llama index integration and query testing

To create a vector store to work with, we'll instantiate milvus lite default server.
In order to have it running we need the following steps:

1. create and define fields to be used in our Milvus collection.
2. Define a schema with set fields.
3. Format the ingest data from a JSON file, dataframe, etc for Milvus readability.
4. Create a collection with a new name and assigning the previously created schema to it.
5. Insert the data.

Afterwards and with the milvus instance running, start a LlamaIndex application from your downloaded index. Use the OpenInferenceCallbackHandler to store your data in OpenInference format, an open standard for capturing and storing AI model inferences that enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Arize and Phoenix.

And finally ask a few questions of your question-answering service and view the responses.

In [7]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"

storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),
)

callback_handler = OpenInferenceCallbackHandler()
service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)

In [8]:
fields = [
      FieldSchema(name="id", dtype=DataType.VARCHAR,
                  is_primary=True, auto_id=False, max_length=65535),
      FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1536),
      FieldSchema(name="doc_id", dtype=DataType.VARCHAR, max_length=65535),
      FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
      FieldSchema(name="node", dtype=DataType.VARCHAR, max_length=65535)]

# define your schema
schema = CollectionSchema(fields=fields,
                          description="this is a collection based on arize docs")

# Transform the data into the format needed for the insert operation
insert_data = [
    [record[field.name] for record in rows] for field in schema.fields
]


In [9]:
milvus_uri = 'https://in03-932fad44bcc2da4.api.gcp-us-west1.zillizcloud.com'
token = '93d42c04f980c5029fafbe36819b80b346b8dce4911e7913a98a2a163307c041cb1beaaf14f5782457fbcc954abb4a6495d8117c'

connections.connect("default", uri=milvus_uri, token=token)
collection_name = "milvus_collection_test"
check_collection = utility.has_collection(collection_name)
if check_collection:
    drop_result = utility.drop_collection(collection_name)
collection = Collection(name=collection_name, schema=schema)

# Insert the data into the collection
collection.insert(insert_data)

(insert count: 1274, delete count: 0, upsert count: 0, timestamp: 443719069233577987, success count: 1274, err count: 0)

In [ ]:



"""
#todo find out a way to do this
vector_store = MilvusVectorStore(
    host='localhost',
    port=default_server.listen_port,
    collection_name="colab_collection",
    overwrite='False'
)

# like this but with serverless Zilliz
vector_store = MilvusVectorStore(
    host=milvus_uri,
    port=403,
    user="db_admin",
    password="foo",
    use_secure=True,
    overwrite='True'
)
"""

index = load_index_from_storage(
    storage_context,
    service_context=service_context,
    vector_store=vector_store
)

In [ ]:

"""

"""


"""
with default_server:
    connections.connect(host='localhost', port=default_server.listen_port)
    collection_name = 'colab_collection'

    # create your collection
    collection = Collection(name=collection_name, schema=schema)

    # Insert the data into the collection
    collection.insert(insert_data)

    vector_store = MilvusVectorStore(
        host='localhost',
        port=default_server.listen_port,
        collection_name="colab_collection",
        overwrite='False'
    )

    index = load_index_from_storage(
        storage_context,
        service_context=service_context,
        vector_store=vector_store
    )

    query_engine = index.as_query_engine()

    max_line_length = 80
    for query in [
        "How do I get an Arize API key?",
        "Can I create monitors with an API?",
        "How do I need to format timestamps?",
        "What is the price of the Arize platform",
    ]:
        print("Query")
        print("=====")
        print()
        print(textwrap.fill(query, max_line_length))
        print()
        response = query_engine.query(query)
        print("Response")
        print("========")
        print()
        print(textwrap.fill(str(response), max_line_length))
        print()

    document_ids, document_texts, document_embeddings = [], [], []
    docstore = storage_context.docstore
    for node_id, node in docstore.docs.items():
        document_ids.append(node.hash)  # use node hash as the document ID
        document_texts.append(node.text)
        expression = "doc_id=='" + str(node_id) + "'"
        vectors = collection.query(
            expr=expression,
            output_fields=['id', 'embedding', 'text', 'node'])
        document_embeddings.append(np.array(vectors[0]['embedding']))
    database_df = pd.DataFrame(
        {
            "document_id": document_ids,
            "text": document_texts,
            "text_vector": document_embeddings,
        }
    )

    database_df.head()

"""

Please double check your internet connection as this may result in the following error: TimeoutError: Milvus not startd in 30.0 seconds

### Loading data into dataframes


To use Phoenix, you must load your data into pandas dataframes.

Your query data is saved in a buffer on the callback handler you defined previously. Load the data from the buffer into a dataframe.


In [ ]:
query_data_buffer = callback_handler.flush_query_data_buffer()
sample_query_df = as_dataframe(query_data_buffer)
list_of_document_id_lists = sample_query_df[":feature.[str].retrieved_document_ids:prompt"].tolist()

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/query_data_complete3.parquet",
)
query_df.head()



In addition to the columns of the previous dataframe, this data has a few additional fields:

    :tag.float:user_feedback: approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)
    :tag.str:openai_relevance_0: a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the first retrieved document is relevant to the query
    :tag.str:openai_relevance_1: a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the second retrieved document is relevant to the query

We'll go over how to compute the relevance classifications in section 6.

Next load your knowledge base into a dataframe.

It is important to remember as we use a milvus vector store, that the connection to the collection needs to be instantiated before being able to query from it.

In [ ]:
database_df = database_df[database_df['text']!='\n']
database_df

# Getting info from the vector store and setting it into a dataframe

The columns of your dataframe are:

- **document_id:** the ID of the chunked document
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text, computed during the LlamaIndex build using "text-embedding-ada-002" from OpenAI

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from OpenAI's "text-embedding-ada-002" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [ ]:
database_embedding_column_name = "text_vector"
database_centroid = database_df[database_embedding_column_name].mean()
database_df[database_embedding_column_name] = database_df[database_embedding_column_name].apply(
    lambda x: x - database_centroid
)
query_embedding_column_name = ":feature.[float].embedding:prompt"
query_centroid = query_df[query_embedding_column_name].mean()
query_df[query_embedding_column_name] = query_df[query_embedding_column_name].apply(
    lambda x: x - query_centroid
)

## 6. Run LLM-Assisted Evaluations

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each retrieved document is relevant to the corresponding query.

💭 Use OpenAI to predict whether each retrieved document is relevant or irrelevant to the query.

⚠️ It's strongly recommended to use GPT-4 for evaluations if you have access.

In [ ]:
evals_model_name = "gpt-3.5-turbo"
# evals_model_name = "gpt-4"  # use GPT-4 if you have access
query_texts = sample_query_df[":feature.text:prompt"].tolist()
list_of_document_id_lists = sample_query_df[":feature.[str].retrieved_document_ids:prompt"].tolist()
document_id_to_text = dict(zip(database_df["document_id"].to_list(), database_df["text"].to_list()))
doc_texts = []
for document_index in [0]:
    for document_ids in list_of_document_id_lists:
        doc_texts.append(document_id_to_text[document_ids[document_index]])

relevance = []
for query_text, document_text in zip(query_texts, doc_texts):
    relevance.append(classify_relevance(query_text, document_text, evals_model_name))

sample_query_df = sample_query_df.assign(
    retrieved_document_text_0=doc_texts,
    relevance_0=relevance,
)
sample_query_df[
    [
        ":feature.text:prompt",
        "retrieved_document_text_0",
        "relevance_0",
    ]
].rename(columns={":feature.text:prompt": "query_text"})

## Compute Ranking Metrics

Now that you know whether each piece of retrieved context is relevant or irrelevant to the corresponding query, you can compute precision@k for k = 1, 2 for each query. This metric tells you what percentage of the retrieved context is relevant to the corresponding query.

precision@k = (# of top-k retrieved documents that are relevant) / (k retrieved documents)

If your precision@2 is greater than zero for a particular query, your LlamaIndex application successfully retrieved at least one relevant piece of context with which to answer the query. If the precision@k is zero for a particular query, that means that no relevant piece of context was retrieved.

Compute precision@k for k = 1, 2 and view the results.

In [ ]:
first_document_relevances = [
    {"relevant": True, "irrelevant": False}.get(rel)
    for rel in query_df[":tag.str:openai_relevance_0"].tolist()
]
second_document_relevances = [
    {"relevant": True, "irrelevant": False}.get(rel)
    for rel in query_df[":tag.str:openai_relevance_1"].tolist()
]

list_of_precisions_at_k_lists = [
    compute_precisions_at_k([rel0, rel1])
    for rel0, rel1 in zip(first_document_relevances, second_document_relevances)
]
precisions_at_1, precisions_at_2 = [
    [precisions_at_k[index] for precisions_at_k in list_of_precisions_at_k_lists]
    for index in [0, 1]
]
query_df[":tag.float:openai_precision_at_1"] = precisions_at_1
query_df[":tag.float:openai_precision_at_2"] = precisions_at_2
query_df[
    [
        ":tag.str:openai_relevance_0",
        ":tag.str:openai_relevance_1",
        ":tag.float:openai_precision_at_1",
        ":tag.float:openai_precision_at_2",
    ]
]

# Launch phoenix

Define your knowledge base dataset with a schema that specifies the meaning of each column (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
import phoenix as px
# get a random sample of 500 documents (including retrieved documents)
# this will be handled by by the application in a coming release
num_sampled_point = 1000
retrieved_document_ids = set(
    [
        doc_id
        for doc_ids in query_df[":feature.[str].retrieved_document_ids:prompt"].to_list()
        for doc_id in doc_ids
    ]
)
retrieved_document_mask = database_df["document_id"].isin(retrieved_document_ids)
num_retrieved_documents = len(retrieved_document_ids)
num_additional_samples = num_sampled_point - num_retrieved_documents
unretrieved_document_mask = ~retrieved_document_mask
sampled_unretrieved_document_ids = set(
    database_df[unretrieved_document_mask]["document_id"]
    .sample(n=num_additional_samples, random_state=0)
    .to_list()
)
sampled_unretrieved_document_mask = database_df["document_id"].isin(
    sampled_unretrieved_document_ids
)
sampled_document_mask = retrieved_document_mask | sampled_unretrieved_document_mask
sampled_database_df = database_df[sampled_document_mask]
sampled_database_df

In [ ]:
database_schema = px.Schema(
    prediction_id_column_name="document_id",
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="text_vector",
        raw_data_column_name="text",
    ),
)
database_ds = px.Dataset(
    dataframe=database_df,
    schema=database_schema,
    name="database",
)

Define your query dataset. Because the query dataframe is in OpenInference format, Phoenix is able to infer the meaning of each column without a user-defined schema by using the phoenix.Dataset.from_open_inference class method.

In [ ]:
query_ds = px.Dataset.from_open_inference(query_df)

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(primary=query_ds, corpus=database_ds)

In [ ]:
session.exports[0]

# Surface Problematic Clusters and Data Points

---



Phoenix helps you:

- visualize your embeddings
- color the resulting point cloud using evaluation metrics
- cluster the points and surface up problematic clusters based on whatever metric you care about

Follow along with the tutorial walkthrough [here](https://youtu.be/hbQYDpJayFw?t=1782), or view the video in your notebook by running the cell below. The video will show you how to investigate your query and knowledge base and identify problematic clusters of data points using Phoenix.

In [ ]:
start_time_in_seconds = int(timedelta(hours=0, minutes=29, seconds=42).total_seconds())
YouTubeVideo("hbQYDpJayFw", start=start_time_in_seconds, width=560, height=315)